# Segmenting & Clustering Toronto Neighborhoods
## Installing Modules

In [1]:
!pip install geocoder

     |████████████████████████████████| 102kB 20.6MB/s ta 0:00:01


In [2]:
!conda install -c conda-forge geopy
from geopy.geocoders import Nominatim

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

## Using Geolocator

In [3]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")

location = geolocator.geocode(address)

latitude = location.latitude
longitude = location.longitude

print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 43.653963, -79.387207.


In [4]:
address = 'M1C'

geolocator = Nominatim(user_agent="ca_explorer")

location = geolocator.geocode(address)

latitude = location.latitude
longitude = location.longitude

print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 50.4215446, 14.9213332628019.


I decided that I'd rather use the csv file instead. 

## Creating a Dataframe from the CSV file

In [5]:
import pandas as pd 
df_coord = pd.read_csv('https://cocl.us/Geospatial_data')
print(df_coord.shape)
df_coord.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Converting "Postal Code" to "Postcode" for consistency
This is so this dataframe can be merged with the scraped wiki dataframe later

In [11]:
df_coord = df_coord.rename(columns={'Postal Code': 'Postcode'}) 
df_coord.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Recreating the table of scraped web data

In [12]:
import pandas as pd
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(url)
df=tables[0]

borough_mask = df.index[df['Borough'] == 'Not assigned']
neighborhood_mask = df.index[df['Neighbourhood'] == 'Not assigned']
neighborhood_and_borough_mask = borough_mask & neighborhood_mask

df.drop(df.index[borough_mask], inplace=True)
df.reset_index(drop=True, inplace=True)

neighborhood_mask = df.index[df['Neighbourhood'] == 'Not assigned']
for idx in neighborhood_mask:
    df['Neighbourhood'][idx] = df['Borough'][idx]
    

f_neighborhoods = lambda x: "%s" % ', '.join(x)
f_boroughs = lambda x: set(x).pop()

temp = df.groupby('Postcode')
temp_neighborhoods = temp['Neighbourhood'].apply(f_neighborhoods)
temp_boroughs = temp['Borough'].apply(f_boroughs)

columns_list = list(zip(temp_boroughs.index, temp_boroughs, temp_neighborhoods))
df_grouped = pd.DataFrame(columns_list)

df_grouped.columns = ['Postcode', 'Borough', 'Neighbourhood']

df_grouped.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## Grouping Tables

In [13]:
df_final = df_grouped.groupby(['Postcode','Borough'], sort=False).agg(lambda x: ','.join(x))
df_final.reset_index(level=['Postcode','Borough'], inplace=True)
print(df_final.shape)
df_final.head(10)

(103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## Merging Tables to create final Dataframe with Latitude and Longitude

In [19]:
df_final_2 = pd.merge(df_final, df_coord, on='Postcode')
print(df_final_2.shape)
df_final_2.head(10)

(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
